In [5]:
import os
import pandas as pd
from dotenv import load_dotenv, find_dotenv

import re
import mechanicalsoup


import moodle_admin.moodle_api as mp

In [2]:
courses = mp.call("core_course_get_courses")
df = pd.DataFrame(courses)

In [3]:
df.head()

,id,shortname,categoryid,categorysortorder,fullname,displayname,idnumber,summary,summaryformat,format,...,groupmodeforce,defaultgroupingid,timecreated,timemodified,enablecompletion,completionnotify,lang,forcetheme,courseformatoptions,hiddensections
0,1,Aularagón,0,1,Aularagón,Aularagón,,,1,site,...,0,0,1488282562,1617710959,0,0,,,"[{'name': 'numsections', 'value': 0}]",NaN
1,3,LIBREOFFICE,43,150014,LibreOffice: La suite ofimática libre,LibreOffice: La suite ofimática libre,,<p>LibreOffice es un software desarrollado y d...,1,topics,...,0,0,1477304039,1552309228,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
2,4,ING-DUOLINGO,58,240008,Inglés Duolingo,Inglés Duolingo,,"<p>La educación personalizada es, cada día más...",1,topics,...,0,0,1488201052,1552047141,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
3,5,Radio,48,210003,La Radio en la Escuela,La Radio en la Escuela,,,1,topics,...,0,0,1487586139,1551439277,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0
4,6,FLAUTA,48,210002,La enseñanza-aprendizaje de la flauta escolar,La enseñanza-aprendizaje de la flauta escolar,,<p></p><p>El presente curso va dirigido a doce...,1,topics,...,0,0,1489408019,1552047305,1,0,,,"[{'name': 'hiddensections', 'value': 0}, {'nam...",0.0


In [10]:
browser = mechanicalsoup.StatefulBrowser()

In [12]:
def required_activities(course_id):
    users_enrolled = mp.call("core_enrol_get_enrolled_users", courseid=course_id)
    test_user_id = [user['id'] for user in users_enrolled if user['roles'][0]["roleid"] == 5][0]
    completion = mp.call(
                "core_completion_get_course_completion_status",
                courseid=course_id,
                userid=test_user_id,
            )
    actividades = [actividades for actividades in completion["completionstatus"]["completions"] if actividades["title"] == "Finalización de actividades"]
    actividades_data = []
    for actividad in actividades:
        pattern = re.compile(r'.*(https://.*edu.es/mod/(\w+)/.*id=(\d+)\">(.*))</a>')
        match = pattern.match(actividad['details']['criteria'])
        item = {
            'url': match[1],
            'type': match[2],
            'id':match[3],
            'course_id':course_id,
            'title':match[4]
        }
        actividades_data.append(item)
    return actividades_data

In [15]:
actividades_a_completar = []
for i in list(courses_ids):
    actividades_a_completar.extend(required_activities(i))
#required_activities(111)

In [16]:
actividades_df = pd.DataFrame(actividades_a_completar)

In [17]:
actividades_df.type.value_counts()

quiz      143
lti        23
forum      12
assign      3
Name: type, dtype: int64

In [18]:
actividades_df[actividades_df['type'] == 'assign']

,url,type,id,course_id,title
0,https://moodle.catedu.es/mod/assign/view.php?i...,assign,35,4,Consigue puntos de experiencia
58,https://moodle.catedu.es/mod/assign/view.php?i...,assign,959,41,Puzzles de code.org
105,https://moodle.catedu.es/mod/assign/view.php?i...,assign,891,55,MODULO 2 UNICA ACTIVIDAD OBLIGATORIA


## Todo este follón es porque las id mostradas en las urls no se corresponden con las ids de las diferentes tablas de la base de datos

In [158]:
for index, row in actividades_df[actividades_df['type'] == 'assign'].iterrows():
    print(f'moosh -n sql-run "UPDATE {row["type"]} set cutoffdate = 1619647140 WHERE title = {row["title"]} and course = {row["course_id"]}"')

moosh -n sql-run "UPDATE assign set cutoffdate = 1619647140 WHERE title = Consigue puntos de experiencia and course = 4"
moosh -n sql-run "UPDATE assign set cutoffdate = 1619647140 WHERE title = Puzzles de code.org and course = 41"
moosh -n sql-run "UPDATE assign set cutoffdate = 1619647140 WHERE title = MODULO 2 UNICA ACTIVIDAD OBLIGATORIA and course = 55"


In [87]:
len(courses_ids)

60

In [88]:
tipos_de_actividad

[{'type': 'assign', 'id': '35'},
 {'type': 'forum', 'id': '36'},
 {'type': 'quiz', 'id': '44'},
 {'type': 'forum', 'id': '45'},
 {'type': 'quiz', 'id': '70'},
 {'type': 'quiz', 'id': '71'},
 {'type': 'quiz', 'id': '72'},
 {'type': 'quiz', 'id': '75'},
 {'type': 'forum', 'id': '88'},
 {'type': 'quiz', 'id': '7013'},
 {'type': 'quiz', 'id': '7015'},
 {'type': 'quiz', 'id': '21709'},
 {'type': 'quiz', 'id': '21712'},
 {'type': 'quiz', 'id': '21718'},
 {'type': 'quiz', 'id': '21723'},
 {'type': 'quiz', 'id': '21733'},
 {'type': 'quiz', 'id': '21740'},
 {'type': 'quiz', 'id': '20242'},
 {'type': 'quiz', 'id': '20243'},
 {'type': 'quiz', 'id': '20235'},
 {'type': 'quiz', 'id': '22749'},
 {'type': 'quiz', 'id': '22750'},
 {'type': 'quiz', 'id': '22751'},
 {'type': 'quiz', 'id': '22752'},
 {'type': 'quiz', 'id': '22753'},
 {'type': 'quiz', 'id': '22754'},
 {'type': 'quiz', 'id': '22755'},
 {'type': 'quiz', 'id': '20313'},
 {'type': 'quiz', 'id': '20314'},
 {'type': 'quiz', 'id': '20315'},
 {'t

In [38]:
completion

{'completionstatus': {'completed': False,
  'aggregation': 2,
  'completions': [{'type': 4,
    'title': 'Finalización de actividades',
    'status': 'No',
    'complete': False,
    'timecompleted': None,
    'details': {'type': 'Finalización de actividades',
     'criteria': '<a href="https://moodle.catedu.es/mod/quiz/view.php?id=70">1.2. Cuestionario</a>',
     'requirement': 'Alcanzando calificación',
     'status': ''}},
   {'type': 4,
    'title': 'Finalización de actividades',
    'status': 'No',
    'complete': False,
    'timecompleted': None,
    'details': {'type': 'Finalización de actividades',
     'criteria': '<a href="https://moodle.catedu.es/mod/quiz/view.php?id=71">2.2. Cuestionario</a>',
     'requirement': 'Alcanzando calificación',
     'status': ''}},
   {'type': 4,
    'title': 'Finalización de actividades',
    'status': 'No',
    'complete': False,
    'timecompleted': None,
    'details': {'type': 'Finalización de actividades',
     'criteria': '<a href="https:

In [14]:
courses_ids = set([
    38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
38,
163,
163,
163,
163,
163,
163,
163,
163,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
57,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
84,
329,
329,
329,
329,
329,
329,
329,
329,
329,
329,
329,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
39,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
168,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
41,
42,
42,
42,
42,
42,
42,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
43,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
4,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
476,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
44,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
6,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
5,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
45,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
46,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
47,
48,
48,
48,
48,
48,
48,
48,
48,
48,
48,
48,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
49,
307,
307,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
167,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
166,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
426,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
165,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
925,
40,
40,
40,
40,
40,
40,
40,
40,
111,
111,
111,
111,
111,
111,
111,
111,
111,
111,
111,
111,
111,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
253,
304,
304,
304,
304,
304,
304,
304,
304,
304,
304,
304,
304,
252,
252,
252,
252,
252,
260,
260,
260,
52,
52,
52,
52,
52,
52,
52,
52,
52,
52,
52,
53,
53,
55,
55,
55,
55,
55,
55,
55,
55,
55,
55,
427,
427,
427,
427,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
923,
56,
56,
56,
56,
56,
56,
56,
56,
56,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
164,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
54,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
171,
171,
171,
171,
171,
171,
171,
171,
171,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
1044,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
160,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
173,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
159,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
174,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
169,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
178,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
179,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1053,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1038,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1039,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1040,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
1041,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
171,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
172,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1051,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1052,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
1043,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
483,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,
1055,])

In [43]:
def get_activities_types(courses_ids):
    

{4,
 5,
 6,
 38,
 39,
 40,
 41,
 42,
 43,
 44,
 45,
 46,
 47,
 48,
 49,
 52,
 53,
 54,
 55,
 56,
 57,
 84,
 111,
 159,
 160,
 163,
 164,
 165,
 166,
 167,
 168,
 169,
 171,
 172,
 173,
 174,
 178,
 179,
 252,
 253,
 260,
 304,
 307,
 329,
 426,
 427,
 476,
 483,
 923,
 925,
 1038,
 1039,
 1040,
 1041,
 1043,
 1044,
 1051,
 1052,
 1053,
 1055}